<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Figure 3: Future projections for each impact

_Authors: Oriana Chegwidden and Jeremy Freeman_

The methods below conduct the analyses to recreate Figure 4 included the
manuscript <<< insert doi >>>. Future risks for fire. We show median changes in
risk with respect to the historic.


In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy as sp
from carbonplan_forest_risks import load, setup, plot, fit, utils, prepare, collect
import xarray as xr
from carbonplan_forest_risks.utils import get_store
import rioxarray

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from showit import image
import scipy as sp
import geopandas as gpd
import regionmask as rm
import altair as alt
alt.themes.enable("carbonplan_light")

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
alt.data_transformers.disable_max_rows()

In [ ]:
coarsen = 4

In [ ]:
mask = (
    (
        load.nlcd(store="az", year=2001).sel(band=[41, 42, 43, 90]).sum("band")
        > 0.25
    )
    .astype("float")
    .coarsen(x=coarsen, y=coarsen, boundary="trim")
    .mean()
)

In [ ]:
historical_fire = xr.open_zarr(
    get_store("carbonplan-scratch", "data/fire_historical_v3.zarr")
).load()
fire_mask = ~np.isnan(historical_fire.historical.isel(time=0).drop("time"))

In [ ]:
ds = (
    xr.open_zarr(get_store("carbonplan-scratch", "data/fire_future_v3.zarr"))
    .assign_coords({"x": mask.x, "y": mask.y})
    .where(fire_mask)
    .groupby("time.year")
    .sum()
    .where(fire_mask)
    .compute()
)

In [ ]:
ds_remaining = (
    xr.open_zarr(
        get_store("carbonplan-scratch", "data/fire_future_v3_remaining.zarr")
    )
    .assign_coords({"x": mask.x, "y": mask.y})
    .groupby("time.year")
    .sum()
    .where(fire_mask)
    .compute()
)

In [ ]:
ds = xr.merge([ds, ds_remaining])

In [ ]:
historical_annual = (
    xr.open_zarr(
        get_store("carbonplan-scratch", "data/fire_historical_v3.zarr")
    )
    .assign_coords({"x": mask.x, "y": mask.y})
    .groupby("time.year")
    .sum()
    .where(fire_mask)
    .mean(dim=["x", "y"])
    .compute()
)

In [ ]:
future_ts = {}
future_ts["fire"] = (
    ds.mean(dim=["x", "y"]).rolling(dim={"year": 10}).mean().compute()
)

In [ ]:
gcms = [
    ("ACCESS-CM2", "r1i1p1f1"),
    ("ACCESS-ESM1-5", "r10i1p1f1"),
    ("MRI-ESM2-0", "r1i1p1f1"),
    ("MIROC-ES2L", "r1i1p1f2"),
    ("MPI-ESM1-2-LR", "r10i1p1f1"),
    ("CanESM5-CanOE", "r3i1p2f1"),
]

### Load in insect data


In [ ]:
insect_url_template = "https://carbonplan.blob.core.windows.net/carbonplan-scratch/from_bill/InsectProjections_3-30/InsectModelProjection_{}.{}.{}-{}.{}-v14climate_3-30-2021.tif"

In [ ]:
insects = load_impacts(
    insect_url_template, historical_fire, coarsen=coarsen
).assign_coords({"year": np.arange(1975, 2100, 10)})
future_ts["insects"] = insects.mean(dim=["x", "y"]).compute() * 100

### Load in drought data


In [ ]:
drought_url_template = "https://carbonplan.blob.core.windows.net/carbonplan-scratch/from_bill/DroughtProjections_3-31/DroughtModelProjection_{}.{}.{}-{}.{}-v14climate_3-30-2021.tif"
drought = load_impacts(
    drought_url_template, historical_fire, coarsen=coarsen
).assign_coords({"year": np.arange(1975, 2100, 10)})
future_ts["drought"] = drought.mean(dim=["x", "y"]).compute() * 100

## Plot the figure


In [ ]:
gcms = [
    ("MRI-ESM2-0", (0, 0)),
    ("MIROC-ES2L", (1, 0)),
    ("MPI-ESM1-2-LR", (2, 0)),
    ("ACCESS-ESM1-5", (3, 0)),
    ("ACCESS-CM2", (4, 0)),
    ("CanESM5-CanOE", (5, 0)),
]
scenario_colors = {
    "ssp245": "#59A82F",
    "ssp370": "#D8B525",
    "ssp585": "#D83232",
}
scenario_colors_light = {
    "ssp245": "#DEEED5",
    "ssp370": "#F7F0D3",
    "ssp585": "#F7D6D6",
}
titles = [
    "Burn area\n[fraction/year]",
    "Drought mortality\n[]",
    "Insect mortality\n[%/year]",
]

In [ ]:
plt.rcParams.update({"font.size": 14, "svg.fonttype": "none"})

fig, axarr = plt.subplots(nrows=3, figsize=(6, 10), sharex=True)

## Fire

for scenario in ["ssp245", "ssp370", "ssp585"]:
    for (gcm, location) in gcms:
        future_ts["fire"]["{}_{}".format(gcm, scenario)].sel(
            year=slice("1970", "2099")
        ).plot(ax=axarr[0], color=scenario_colors_light[scenario], alpha=1)

for scenario in ["ssp245", "ssp370", "ssp585"]:
    future_ts["fire"][
        ["{}_{}".format(gcm[0], scenario) for gcm in gcms]
    ].to_array(dim="vars").mean("vars").plot(
        ax=axarr[0], color=scenario_colors[scenario], label=scenario
    )
historical_annual["historical"].rolling(dim={"year": 10}).mean().plot(
    ax=axarr[0], color="k", linestyle="-", label="Historical"
)
axarr[0].set_xlabel("")
axarr[0].set_ylim(0, 0.015)
axarr[0].set_ylim(0, 0.015)

axarr[0].legend()

## Insects

for scenario in ["ssp245", "ssp370", "ssp585"]:
    for (gcm, location) in gcms:
        future_ts["insects"].sel(scenario=scenario)[gcm].sel(
            year=slice(1970, 2099)
        ).plot(ax=axarr[2], color=scenario_colors_light[scenario], alpha=1)

for scenario in ["ssp245", "ssp370", "ssp585"]:
    future_ts["insects"].sel(scenario=scenario).to_array(dim="vars").mean(
        "vars"
    ).plot(ax=axarr[2], color=scenario_colors[scenario], label=scenario)

## Drought

for scenario in ["ssp245", "ssp370", "ssp585"]:
    for (gcm, location) in gcms:
        future_ts["drought"].sel(scenario=scenario)[gcm].sel(
            year=slice(1970, 2099)
        ).plot(
            ax=axarr[1], color=scenario_colors_light[scenario], alpha=1
        )  # , alpha=0.5, #marker='.', linestyle='None'

for scenario in ["ssp245", "ssp370", "ssp585"]:
    future_ts["drought"].sel(scenario=scenario).to_array(dim="vars").mean(
        "vars"
    ).plot(
        ax=axarr[1],
        color=scenario_colors[scenario],
        label=scenario,  # marker='.', markersize=12, linestyle='None'
    )

for i, impact in enumerate(titles):
    axarr[i].set_ylabel(impact)
impacts = ["Fire", "Drought", "Insects"]
for i, impact in enumerate(impacts):
    axarr[i].set_title(impact)
plt.tight_layout()
plt.savefig("future_traces.svg", format="svg")